In [ ]:
R.home()

In [ ]:
baizer::pkglib(tidyverse, baizer)

In [ ]:
sample <- snakemake@wildcards[['sample']]

In [ ]:
path <- str_c(snakemake@input[['count_dir']], snakemake@config[['count_VDJB_airr']], sep='/')
path <- str_glue(path)

In [ ]:
cr_airr <- readr::read_tsv(path)

In [ ]:
nt2aa <- function(x) {
  ds <- Biostrings::DNAStringSet(x)
  res <- Biostrings::translate(ds,
    no.init.codon = TRUE, if.fuzzy.codon = "solve"
  ) %>%
    as.character()
  return(res)
}

In [ ]:
parse_CellRanger_vdjseq <- function(x) {
  if (any(c(
    "cell_id", "sequence_id", "sequence",
    "v_sequence_start", "sequence_aa"
  ) %nin% colnames(x))) {
    stop("please read from airr_rearrangement.tsv!")
  }

  x <- x %>% dplyr::mutate(
      seq_orf_nt = str_sub(.data[["sequence"]], .data[["v_sequence_start"]])
    ) %>% dplyr::mutate( seq_orf_aa = nt2aa(.data[["seq_orf_nt"]]))

  if (any(x[["seq_orf_aa"]] != x[["sequence_aa"]])) {
    stop("please check the sequences!")
  }

  if (any(!str_detect(x[["seq_orf_aa"]], "^M"))) {
    stop("please check the sequences!")
  }

  res <- x[, c(
    "cell_id", "sequence_id",
    "sequence", "seq_orf_nt", "seq_orf_aa"
  )]

  return (res)
}


In [ ]:
write_fasta <- function(x, file) {
  if (is.null(names(x))) {
    names(x) <- seq_along(x)
  }

  res <- map2_chr(names(x), x, ~ str_c(">", .x, "\n", .y))
  out <- file(file, "w")
  writeLines(res, out)
  close(out)
}

In [ ]:
cr_airr <- parse_CellRanger_vdjseq(cr_airr)

In [ ]:
cr_airr %>% pull(seq_orf_nt, sequence_id) %>% 
    write_fasta(snakemake@output[['VDJB_orf_nt_fa']])

In [ ]:
cr_airr %>% pull(seq_orf_aa, sequence_id) %>% 
    write_fasta(snakemake@output[['VDJB_orf_aa_fa']])